In [ ]:
# !pip install nltk gdown seaborn torchtext pymorphy2 gensim

In [ ]:
import random
import numpy as np
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Семинар: Language Model

Привет! Сегодня мы создадим свою Language Model! Посмотрим на три вида моделей: N-gram, CNN, LSTM. Для обучения LM лучше всего подходят большие корпуса с разнообразными текстами: от новостей до художственной литературы. Для русского языка есть большой корпус [Taiga](https://tatianashavrina.github.io/taiga_site/). Для английского используют тексты из [википедии](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/) или [BookCorpus](https://github.com/soskek/bookcorpus). 

Сегодня вы возьмем маленькую часть датасета Taiga: новости с сайта [nplus1](https://nplus1.ru). Каждая новость на сайте помечается меткой сложности (от 0 до 10). Это не поможет нам с обучением хорошей LM, но даст возможность поиграться с генерацией текста.

Загрузим датасет и подготовим его к работе!

In [ ]:
import gdown


gdown.download("https://drive.google.com/uc?id=1UtF9urwAL2OiMg7N5iFmZmeiRzq1Psw6")

In [ ]:
!unzip nplus1.zip

In [ ]:
!ls nplus1/

Вся информация про тексты содержится в таблице `newmetadata.csv`. Загрузим её с помощью `pandas`.

In [ ]:
import pandas as pd


metadata = pd.read_table("nplus1/newmetadata.csv")
metadata.head()

Колонка `textdiff` содержит информацию про сложность текста. Чтобы выделить нужный кусок, воспользуемся методами `pandas`.

In [ ]:
metadata[(metadata["textdiff"] > 4) & (metadata["textdiff"] < 5)].shape

Посмотрим на распределение сложности текстов:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


_, ax = plt.subplots(1, 1, figsize=(10, 5))
sns.set()
sns.histplot(metadata["textdiff"], ax=ax)

Загрузим предобученные эмбеддинги, которые готовы к работе с русским языком ([весь список](https://rusvectores.org/ru/models/)). Из-за особенностей русского языка эмбеддинги ожидают строку вида `{слово}_{часть речи}`. Надо про это помнить при работе с этими эмбеддингами.

In [ ]:
from gensim import downloader as api

word2vec = api.load('word2vec-ruscorpora-300')

Подготовим датасет к работе с моделями:

In [ ]:
from pathlib import Path
import string

import numpy as np
from tqdm.notebook import tqdm

import nltk
from pymorphy2 import MorphAnalyzer

from torch.utils.data import Dataset, random_split


PAD = "<PAD>"
EOS = "<EOS>"
UNK = "<UNK>"


class TextDataset(Dataset):
    def __init__(self, min_diff=0, max_diff=10):
        self.root = Path("nplus1/texts")
        metadata = pd.read_table("nplus1/newmetadata.csv")
        self.metadata = ... # Получи нужную часть таблицы с помощью `min_diff` и `max_diff`
        
        # Получим список всех текстов и сверим его с таблицей
        file_paths = np.array(list(self.root.glob("*.txt")))
        text_ids = np.array(list(path.name.split(".")[0] for path in file_paths))
        self.text_ids = text_ids[np.isin(text_ids, self.metadata["textid"])]
        self.file_paths = file_paths[np.isin(text_ids, self.metadata["textid"])]
        
        self.min_diff = min_diff
        self.max_diff = max_diff
        
        self.tokenizer = nltk.WordPunctTokenizer()
        self.morph = MorphAnalyzer()
        
        self.token2idx = {PAD: 0, EOS: 1, UNK: 2}
        self.vocab = set([PAD, EOS, UNK])
        for path in tqdm(self.file_paths):
            with open(path) as file:
                text = ...# прочитай текст из файла
                self.vocab.update(...) # добавь токены в словарь
        self.token2idx.update({t:num + 3 for num, t in enumerate(vocab)})
        self.idx2token = {num: token for token, num in self.token2idx.items()}
            
    
    def __getitem__(self, item):
        with open(self.file_paths[item]) as file:
            text = file.read()
            
        tokens = ... # с токенизируй текст
        
        text_id = self.text_ids[item]
        textdiff = ... # получи сложность текста
        
        # для обучение нейронок нам потребуются индексы токенов в словаре.
        input_ids = ... # получи их из self.token2idx
        
        return {
            "text": text,
            "tokens": tokens,
            "textdiff": textdiff,
            "input_ids": input_ids
        }
    
    def __len__(self):
        return len(self.file_paths)
    
    def tokenize_(self, text):
        tokens = self.tokenizer.tokenize(text.lower())
        morphs = [self.morph.parse(token)[0]
                  for token in tokens 
                  if (token not in string.punctuation)]
        tokens = [f"{morph.normal_form}_{morph.tag.POS}" for morph in morphs]
        tokens = [token for token in tokens if token in word2vec]
        tokens += [EOS]
        return tokens
    
    def embeddins(self):
        w = torch.rand(len(self.vocab), word2vec.vector_size)
        for token, num in self.token2idx.items():
            if token in word2vec:
                w[num] = ... # получи эмбеддинг для токена
        return w

In [ ]:
dataset = TextDataset(0.5, 2)
train_size = np.ceil(len(dataset) * 0.8).astype(int)


train_dataset, valid_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

## 1. N-gram LM

Первая жертва – N-граммная модель. Она пишется скучно, но хорошо работает.

In [ ]:
from collections import Counter, defaultdict

from tqdm.notebook import tqdm


class NGramModel(object):
    '''
    Структура этой реализации n-граммной модели следующая:
    self.ngrams – словарь, который на каждый (token_0, ..., token_(n-1)) – n-1 tuple из токенов
        хранит частоту появления следующего токена. Для подсчета числа токенов воспользуемся
        Counter
    self.tokenize_func – функция токенизации текста. С её помощью будем получать токены.
    '''
    def __init__(self, n=2):
        self.ngrams = defaultdict(Counter)
        self.n = n
        self.tokenize_func = None
        
    def compute_ngrams(self, dataset, tokenize_func):
        self.tokenize_func = tokenize_func
        self.ngrams = defaultdict(Counter)
        for row in tqdm(dataset):
            ngram = [PAD] * self.n
            for token in row["tokens"]:
                ... # обнови self.ngram новыми токенами
            
    def get_log_probs(self, prefix, min_log_pr=-15):
        '''
        Функция, которая будет возвращать логарифмы частот появления токенов
        '''
        if isinstance(prefix, str):
            # преврати строку в tuple из токенов с помощью tokenize_func. 
            prefix = ... # не забывай, что tokenize_func добавляет <EOS>
        if len(prefix) < self.n - 1:
            prefix = [PAD] * (self.n - len(prefix) - 1) + prefix
        else:
            prefix = prefix[-self.n + 1:]
        possible_ends = ... # получи количество появление токенов с таким префиксом
        sum_freq = ... # получи количество появление префикса в текстах
        return ... # верни логарифм частоты появления токенов
    
    def sample(self, prefix):
        possible_ends = self.get_log_probs(prefix)
        if len(possible_ends) > 0:
            end = np.random.choice(list(possible_ends.keys()), p=np.exp(list(possible_ends.values())))
            return end
        return EOS

Создадим 5-граммную модель и посмотрим, как хорошо справляется она с генерацией текста.

In [ ]:
frigram = NGramModel(5)

In [ ]:
frigram.compute_ngrams(train_dataset, dataset.tokenize_)

In [ ]:
frigram.get_log_probs("")

In [ ]:
frigram.sample("")

In [ ]:
def generate_text(model, prefix, lenth=100):
    text = "" + prefix
    while len(text) < lenth:
        ... # получи новый токен по предыдущим. Добавь его в текст.
    return text

In [ ]:
generate_text(frigram, "")

Количественная величина, которая позволяет сравнивать LM: перплекция. Для её вычисления используется следующая формула:

$$
\text{Ppr} = \frac{1}{|D|} \sum_{t \in D}\sum_{w \in t} - \log (p(w)),
$$
где $D$ – валидационный датасет, $|D|$ – общая длина текстов.

In [ ]:
def perplexity_ngram(dataset, model):
    lengths = 0
    log_prob = 0
    for row in tqdm(dataset):
        ... # получи метрики для вычисление перплексии для текущей сторки
    return np.exp(-log_prob / lengths)

In [ ]:
perplexity_ngram(valid_dataset, frigram)

## 2. NN LM

Приступим к нейросетевым языковым моделям. Для начала нам потребуется сэмплер из прошлого семинара.

In [ ]:
from torch.utils.data import Sampler


class TextSampler(Sampler):
    def __init__(self, sampler, batch_size_tokens=1e4):
        self.sampler = sampler
        self.batch_size_tokens = batch_size_tokens

    def __iter__(self):
        batch = []
        max_len = 0
        for ix in self.sampler:
            row = self.sampler.data_source[ix]
            max_len = max(max_len, len(row["input_ids"]))
            if (len(batch) + 1) * max_len > self.batch_size_tokens:
                yield batch
                batch = []
                max_len = len(row["input_ids"])
            batch.append(ix)
        if len(batch) > 0:
            yield batch

    def __len__(self):
        return len(self.sampler)

In [ ]:
def collate_fn(batch):
    max_len = max(len(row["input_ids"]) for row in batch)
    input_embeds = np.zeros((len(batch), max_len))
    for idx, row in enumerate(batch):
        input_embeds[idx][:len(row["input_ids"])] += row["input_ids"]
    row["input_ids"] = torch.LongTensor(input_embeds)
    return row

In [ ]:
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler, random_split


train_sampler = RandomSampler(train_dataset)
valid_sampler = SequentialSampler(valid_dataset)

train_loader = DataLoader(train_dataset, batch_sampler=TextSampler(train_sampler), collate_fn=collate_fn, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_sampler=TextSampler(valid_sampler), collate_fn=collate_fn, num_workers=4)

### CNN

Вторая жертва – CNN. Если внимательно посмотреть, то она является нейросетевым приближением к n-грамной модели. Для её реализации нам потребуется новым модуль – `nn.ZeroPad2d`[docs](https://pytorch.org/docs/stable/generated/torch.nn.ZeroPad2d.html). Он добавит нулей в нужном месте, чтобы конволюционный слой смотрел только на предыдущие токены при предсказании текущего.

In [ ]:
import torch
import torch.nn as nn


class CNNLM(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        
        self.emb = nn.Embedding(vocab_size, emb_size)
        self.pad = ... # создай ZeroPad2d
        self.conv = ... # создай однослойную конволюцию
        self.pred = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, input_ids):
        embed = self.emb(input_ids).permute(0, 2, 1)
        padded = self.pad(embed)
        convolved = torch.relu(self.conv(padded)).permute(0, 2, 1)
        return self.pred(convolved)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNNLM(len(dataset.vocab), 300, 100).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss(ignore_index=dataset.token2idx[PAD])

In [ ]:
with torch.no_grad():
    model.emb.weight.copy_(dataset.embeddins())

In [ ]:
num_epochs = 1

for e in range(num_epochs):
    model.train()
    with tqdm(total=len(train_loader)) as pbar:
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)

            ... # обучи модель 

            pbar.update(input_ids.size(0))
        
    model.eval()
    valid_loss = 0
    n_iter = 0
    with torch.no_grad():
        for batch in valid_loader:
            n_iter += 1
            input_ids = batch["input_ids"].to(device)
            prediction = model(input_ids[:, :-1])
            valid_loss += criterion(prediction.reshape(-1, prediction.size(-1)), input_ids[:, 1:].reshape(-1))
    print(f"Valid Loss: {valid_loss / n_iter}, Valid Peprplexity: {torch.exp(valid_loss / n_iter)}")

После обучения модели посмотрим, как она справляется с задачей генерации текста. Сделаем специальную функцию для этого.

In [ ]:
def sample(model, prefix, max_length=100):
    tokens = dataset.tokenize_(prefix)[:-1]
    input_ids = [dataset.token2idx.get(token) for token in tokens]
    input_ids_tensor = torch.LongTensor(input_ids).unsqueeze(0).to(device)
    
    with torch.no_grad():
        while True:
            output = model(input_ids_tensor)

            probs = ... # получи из output вероятности следующего токена
            next_id = ... # получи следующий токен
            tokens += ... # добавь токен в список токенов
            
            if dataset.idx2token[next_id] == EOS or len(tokens) > max_length:
                break
            input_ids += [next_id]
            input_ids_tensor = torch.LongTensor(input_ids).unsqueeze(0).to(device)
    
    return " ".join(t.split("_")[0] for t in tokens)

In [ ]:
model.eval()
sample(model, "привет")

### LSTM

Последняя жертва – LSTM. Она должна лучше работать с длинными текстами, потому что у неё нет фиксированного количества токенов, на которые она можеть "смотреть".

In [ ]:
import torch
import torch.nn as nn


class LSTMLM(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        
        self.emb = nn.Embedding(vocab_size, emb_size)
        self.lstm = ... # Сделай lstm слой
        self.pred = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, input_ids):
        embs = self.emb(input_ids)
        output, _ = self.lstm(embs)
        return self.pred(output)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = LSTMLM(len(dataset.vocab), 300, 100).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss(ignore_index=dataset.token2idx[PAD])

In [ ]:
with torch.no_grad():
    model.emb.weight.copy_(dataset.embeddins())

In [ ]:
num_epochs = 1

for e in range(num_epochs):
    model.train()
    with tqdm(total=len(train_loader)) as pbar:
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)

            ... # обучи модель 

            pbar.update(input_ids.size(0))
        
    model.eval()
    valid_loss = 0
    n_iter = 0
    with torch.no_grad():
        for batch in valid_loader:
            n_iter += 1
            input_ids = batch["input_ids"].to(device)
            prediction = model(input_ids[:, :-1])
            valid_loss += criterion(prediction.reshape(-1, prediction.size(-1)), input_ids[:, 1:].reshape(-1))
    print(f"Valid Loss: {valid_loss / n_iter}, Valid Peprplexity: {torch.exp(valid_loss / n_iter)}")

In [ ]:
model.eval()
sample(model, "привет")

### Что дальше?
 
Если мы говорим про генерацию, то модель надо обучать подольше и на большом количестве текста. Если хочешь поэкспериментировать с генерацией, то я предлагаю такой план:
 
- обучи модель на всех новостях (min_diff=0, max_diff=10)
- сохрани веса этой модели (torch.save(model.state_dict()))
- переобучи несколько моделей на новостях с другими значениями сложности (eg. (min_diff=1, max_diff=3), (min_diff=4, max_diff=8))
- сравни сгенерированные тексты